<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [2]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding
from tensorflow.keras.datasets import imdb

In [0]:
# TODO - Words, words, mere words, no matter from the heart.
import requests
url = "https://www.gutenberg.org/files/100/100-0.txt"
w = requests.get(url)
w.encoding = w.apparent_encoding 
data = w.text

In [4]:
data[:1500]

'\r\nProject Gutenberg’s The Complete Works of William Shakespeare, by William\r\nShakespeare\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever.  You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org.  If you are not located in the United States, you’ll\r\nhave to check the laws of the country where you are located before using\r\nthis ebook.\r\n\r\n\r\nTitle: The Complete Works of William Shakespeare\r\n\r\nAuthor: William Shakespeare\r\n\r\nRelease Date: January 1994 [EBook #100]\r\nLast Updated: November 7, 2019\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***\r\n\r\n\r\n\r\n\r\nThe Complete Works of William Shakespeare\r\n\r\n\r\n\r\nby William Shakespeare\r\n\r\

In [5]:
len(data)

5740053

In [0]:
#Clean up the data 
text = data.replace('\r', '')
'''Lower casing text '''
text = data[900:-2500].lower()
'''fixing the spacing issue '''
text = ' '.join(data.split())


In [0]:
chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

In [8]:
len(chars)

104

In [9]:
#create sequence data:
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences =  []
next_chars = []

for i in range(0, len(encoded) - maxlen, step):
  sequences.append(encoded[i : i + maxlen])
  next_chars.append(encoded[i + maxlen])

print(len(sequences))

1055988


In [0]:
#Look at sequences
sequences[:5]

In [10]:
next_chars[:5]

[65, 53, 96, 84, 92]

In [0]:
#specify x & y
x = np.zeros((len(sequences), maxlen, len(chars)),dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
  for t, char in enumerate(sequence):
    x[i,t,char] = 1

    y[i, next_chars[i]] = 1


In [12]:
x.shape, y.shape

((1055988, 40, 104), (1055988, 104))

In [13]:
x[0].shape

(40, 104)

In [14]:
print(x[0][0].shape)
x[0][0]

(104,)


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [15]:
y[0] #just realized we are using last 40 words (x) as input to predict next letter (y)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
def sample(y_pred, temperature=1.0):
  #help function to sample an index from a probability array
  y_pred = np.asarray(y_pred).astype('float64')
  y_pred = np.log(y_pred) / temperature
  exp_pred = np.exp(y_pred)
  y_pred = exp_pred / np.sum(exp_pred)
  prob = np.random.multinomial(1, y_pred, 1)
  return np.argmax(prob)

In [0]:
def on_epoch_end(epoch, _):
  #function invoked at end of each epoch. Prints generated text
  print()
  print('----- Generating text after Epoch: %d' % epoch)

  start_index = random.randint(0,len(text) - maxlen - 1)
  for diversity in [0.2, 0.5, 1.0, 1.2]:
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
        x_pred[0, t, char_int[char]] = 1

      y_pred = model.predict(x_pred, verbose=0)[0]
      next_index = sample(y_pred, diversity)
      next_char = int_char[next_index]

      sentence = sentence[1:] + next_char

      sys.stdout.write(next_char)
      sys.stdout.flush()
    print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [18]:
#fit the model and set output to history var
history = model.fit(x,
                    y,
                    batch_size=128,
                    epochs=5
    
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1055988 samples
Epoch 1/5
1055988/1055988 [==============================] - 472s 447us/sample - loss: 2.2483
Epoch 2/5
1055988/1055988 [==============================] - 442s 419us/sample - loss: 1.9316
Epoch 3/5
1055988/1055988 [==============================] - 434s 411us/sample - loss: 1.8228
Epoch 4/5
1055988/1055988 [==============================] - 430s 407us/sample - loss: 1.7557
Epoch 5/5
1055988/1055988 [==============================] - 427s 405us/sample - loss: 1.7107


# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN